# VacationPy

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,ust-nera,64.5431,143.1150,288.29,55,29,2.60,1720403376,Russia
1,1,iskateley,67.6514,53.0783,290.42,62,22,3.90,1720403379,Russia
2,2,plunge,55.8955,21.8015,283.46,87,92,1.29,1720403381,Lithuania
3,3,port-aux-francais,-49.3649,70.2099,275.45,83,90,13.16,1720403384,France
4,4,matoury,4.7382,-52.4119,297.92,100,0,0.51,1720403386,France


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
# Create a dictionary containing coordinates
coordinates = {
    "latitude": city_data_df['Lat'],
    "longitude": city_data_df['Lng'],
    "City": city_data_df['City'],
    "Country": city_data_df['Country'],
    "Humidity": city_data_df['Humidity']
}

# Create a Pandas DataFrame containing coordinates
coordinates_df = pd.DataFrame(coordinates)

# Display sample data
coordinates_df

,latitude,longitude,City,Country,Humidity
0,64.5431,143.1150,ust-nera,Russia,55
1,67.6514,53.0783,iskateley,Russia,62
2,55.8955,21.8015,plunge,Lithuania,87
3,-49.3649,70.2099,port-aux-francais,France,83
4,4.7382,-52.4119,matoury,France,100
...,...,...,...,...,...
599,14.4290,-88.7414,gracias,Honduras,97
600,-17.3255,-68.1085,patacamaya,Bolivia,56
601,-24.6467,-50.2100,pirai do sul,Brazil,73
602,40.3776,-104.9074,greeley,United States,75


In [6]:
%%capture --no-display

# Configure the map plot
map_plot_1 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.3,
    color = "City",
    hover_cols = 'all'
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (City,Humidity,index,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[city_data_df['Humidity'] < 65].reset_index()

# Drop any rows with null values
ideal_weather_df = city_data_df.dropna(how='any')

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,ust-nera,64.5431,143.1150,288.29,55,29,2.60,1720403376,Russia
1,1,iskateley,67.6514,53.0783,290.42,62,22,3.90,1720403379,Russia
2,2,plunge,55.8955,21.8015,283.46,87,92,1.29,1720403381,Lithuania
3,3,port-aux-francais,-49.3649,70.2099,275.45,83,90,13.16,1720403384,France
4,4,matoury,4.7382,-52.4119,297.92,100,0,0.51,1720403386,France
...,...,...,...,...,...,...,...,...,...,...
599,599,gracias,14.4290,-88.7414,292.23,97,100,0.79,1720404655,Honduras
600,600,patacamaya,-17.3255,-68.1085,278.35,56,100,3.48,1720404657,Bolivia
601,601,pirai do sul,-24.6467,-50.2100,287.41,73,99,2.02,1720404658,Brazil
602,602,greeley,40.3776,-104.9074,291.15,75,0,4.12,1720404660,United States


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ust-nera,Russia,64.5431,143.1150,55,
1,iskateley,Russia,67.6514,53.0783,62,
2,plunge,Lithuania,55.8955,21.8015,87,
3,port-aux-francais,France,-49.3649,70.2099,83,
4,matoury,France,4.7382,-52.4119,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ust-nera - nearest hotel: No hotel found
iskateley - nearest hotel: No hotel found
plunge - nearest hotel: Beržas
port-aux-francais - nearest hotel: Keravel
matoury - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
nova vicosa - nearest hotel: No hotel found
cole harbour - nearest hotel: Westin Nova Scotian
papatowai - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
isla mujeres - nearest hotel: Isla Mujeres Palace Hotel
bredasdorp - nearest hotel: Victoria Hotel
kuta - nearest hotel: The Patra Jasa Bali Resort & Villas
las heras - nearest hotel: No hotel found
college - nearest hotel: Odessa Marriott Hotel & Conference Center
vilyuchinsk - nearest hotel: Вилючинская
kingston - nearest hotel: No hotel found
tura - nearest hotel: Hôtel Chantepie
codrington - nearest hotel: Tracy Park Hotel and Country Club
puerto madryn - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
blackmans bay - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ust-nera,Russia,64.5431,143.1150,55,No hotel found
1,iskateley,Russia,67.6514,53.0783,62,No hotel found
2,plunge,Lithuania,55.8955,21.8015,87,Beržas
3,port-aux-francais,France,-49.3649,70.2099,83,Keravel
4,matoury,France,4.7382,-52.4119,100,No hotel found
...,...,...,...,...,...,...
599,gracias,Honduras,14.4290,-88.7414,97,No hotel found
600,patacamaya,Bolivia,-17.3255,-68.1085,56,No hotel found
601,pirai do sul,Brazil,-24.6467,-50.2100,73,Pousada do Canyon Guartelá
602,greeley,United States,40.3776,-104.9074,75,Comfort Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
# Create a dictionary containing coordinates for ideal weather
coordinates = {
    "latitude": hotel_df['Lat'],
    "longitude": hotel_df['Lng'],
    "City": hotel_df['City'],
    "Hotel": hotel_df['Hotel Name'],
    "Humidity": hotel_df['Humidity']
}

# Create a Pandas DataFrame containing coordinates
coordinates_df = pd.DataFrame(coordinates)

# Display sample data
coordinates_df

,latitude,longitude,City,Hotel,Humidity
0,64.5431,143.1150,ust-nera,No hotel found,55
1,67.6514,53.0783,iskateley,No hotel found,62
2,55.8955,21.8015,plunge,Beržas,87
3,-49.3649,70.2099,port-aux-francais,Keravel,83
4,4.7382,-52.4119,matoury,No hotel found,100
...,...,...,...,...,...
599,14.4290,-88.7414,gracias,No hotel found,97
600,-17.3255,-68.1085,patacamaya,No hotel found,56
601,-24.6467,-50.2100,pirai do sul,Pousada do Canyon Guartelá,73
602,40.3776,-104.9074,greeley,Comfort Suites,75


In [13]:
%%capture --no-display

# Configure the map plot
map_plot_2 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.3,
    color = "City",
    hover_cols='all'
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [longitude,latitude]   (City,Humidity,index,Hotel)